In [66]:
import os
import ast
from dotenv import load_dotenv
import google.genai as genai
from google.genai import Client
from google.genai.types import Tool, GoogleSearchRetrieval

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

In [67]:
def planner_agent(client: Client, topic: str) -> list[str]:
    print("Planner Agent: Creating a research plan...")
    prompt = f"""
    You are an expert research planner. Your task is to break down the following topic
    into 3-5 specific, answerable questions. Return these questions as a strict Python list of strings.
    Do not include any other text, explanation, or markdown formatting.
    Just return the list.

    TOPIC: "{topic}"

    Example output: ["question 1", "question 2", "question 3"]
    """
    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt
        )
        plan_str = response.text.strip()

        # 🌟 DEBUGGING: Print the raw response from the model
        print(f"Planner Agent: Raw model output:\n---\n{plan_str}\n---")

        # Now, try to parse it
        plan = ast.literal_eval(plan_str)

        if not isinstance(plan, list):
            raise ValueError("Model did not return a list.")

        print("Plan created:")
        for i, q in enumerate(plan, 1):
            print(f"   {i}. {q}")
        return plan
    except Exception as e:
        print(f"Error in Planner Agent: {e}")
        return []

In [71]:
def search_agent(client: Client, question: str) -> str:
    print(f"Search Agent: Researching question: '{question}'...")
    try:
        search_tool = Tool(
            google_search_retrieval=GoogleSearchRetrieval()
        )
        prompt = f"Provide a detailed, concise answer to the following question: {question}"
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt,
        )
        print("   - Information found.")
        return response.text
    except Exception as e:
        print(f"Error in Search Agent: {e}")
        return ""


In [72]:
def synthesizer_agent(client: Client, topic: str, research_results: list) -> str:
    print("Synthesizer Agent: Writing the final report...")
    research_notes = ""
    for question, data in research_results:
        research_notes += f"### Question: {question}\n### Research Data:\n{data}\n\n---\n\n"

    prompt = f"""
    You are an expert research analyst. Your task is to synthesize the provided research notes
    into a comprehensive, well-structured report on the topic: "{topic}".
    The report should have an introduction, a body that covers the key findings from the notes,
    and a conclusion. Use the information from the research notes ONLY. Format the report using Markdown.

    ## Research Notes ##
    {research_notes}
    """
    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt
        )
        return response.text
    except Exception as e:
        print(f"Error in Synthesizer Agent: {e}")
        return "Error: Could not generate the final report."

In [73]:
def main():
    try:
        client = genai.Client()
    except Exception as e:
        print(f"Error initializing AI client: {e}")
        print("Please ensure your GOOGLE_API_KEY is set and the SDK is installed.")
        return

    print("\nHello! I am your AI Research Assistant.")
    topic = input("What topic would you like me to research today? ")

    if not topic.strip():
        print("A topic is required to begin research. Exiting.")
        return

    print(f"\nStarting research process for: '{topic}'")
    research_plan = planner_agent(client, topic)
    if not research_plan:
        print("Could not create a research plan. Exiting.")
        return

    research_results = []
    print("\n--- Starting Research Execution ---")
    for question in research_plan:
        research_data = search_agent(client, question)
        if research_data:
            research_results.append((question, research_data))

    if not research_results:
        print("\nCould not find any information during research. Exiting.")
        return

    print("\n--- Synthesizing Final Report ---")
    final_report = synthesizer_agent(client, topic, research_results)

    print("\n\n--- FINAL RESEARCH REPORT ---")
    print(f"## Topic: {topic}\n")
    print(final_report)
    print("--- END OF REPORT ---")

if __name__ == "__main__":
    main()




Hello! I am your AI Research Assistant.

Starting research process for: 'weather'
Planner Agent: Creating a research plan...
Planner Agent: Raw model output:
---
["What are the primary atmospheric variables and processes that define and create local weather conditions?", "How are various weather phenomena, such as temperature, precipitation, and wind, measured and forecast?", "What are the major impacts of extreme weather events on human infrastructure and natural ecosystems?", "How does climate change influence the frequency, intensity, and distribution of global weather patterns?"]
---
Plan created:
   1. What are the primary atmospheric variables and processes that define and create local weather conditions?
   2. How are various weather phenomena, such as temperature, precipitation, and wind, measured and forecast?
   3. What are the major impacts of extreme weather events on human infrastructure and natural ecosystems?
   4. How does climate change influence the frequency, intens